In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [33]:
# Load your dataset
df = pd.read_csv("excel_action_log.csv")

# Encode actions as strings
df["Action"] = df["Action Type"] + ":" + df["Key/Button"]

In [34]:
# Sliding window of 4 actions
window_size = 4
sequences, intents = [], []

for i in range(len(df) - window_size):
    seq = df.iloc[i:i+window_size]
    if len(seq["Intent"].unique()) == 1:
        action_seq = seq["Action"].tolist()
        sequences.append(action_seq)
        intents.append(seq["Intent"].iloc[-1])

In [35]:
# Encode actions
action_encoder = LabelEncoder()
flat_actions = [action for seq in sequences for action in seq]
action_encoder.fit(flat_actions)
encoded_sequences = [[action_encoder.transform([a])[0] for a in seq] for seq in sequences]

In [36]:
# Pad sequences
X = pad_sequences(encoded_sequences, maxlen=window_size, padding='pre')
y = to_categorical(encoded_intents)

In [37]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
# LSTM Model
vocab_size = len(action_encoder.classes_)
num_classes = y.shape[1]

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=window_size),
    LSTM(64),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [39]:
# Train
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 1.9463 - val_accuracy: 0.2500 - val_loss: 1.9348
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.5926 - loss: 1.9306 - val_accuracy: 0.2500 - val_loss: 1.9232
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.7778 - loss: 1.9147 - val_accuracy: 0.5000 - val_loss: 1.9113
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.8889 - loss: 1.8985 - val_accuracy: 0.5000 - val_loss: 1.8991
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.8889 - loss: 1.8817 - val_accuracy: 0.5000 - val_loss: 1.8864
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.8889 - loss: 1.8642 - val_accuracy: 0.5000 - val_loss: 1.8730
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.8889 - loss: 1.8456 - val_accuracy: 0.5000 - val_loss: 1.8589
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.8889 - loss: 1.8259 - val_accuracy: 0.5000 - val_loss: 

In [40]:
# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 1.0000 - loss: 1.7866
Test Accuracy: 1.00
